In [20]:
import logging
import os
import pickle
import sys
import time
from pathlib import Path
from statistics import mean

import numpy as np
import polars as pl
import yaml
from sklearn.calibration import LabelEncoder
from sklearn.preprocessing import MinMaxScaler


In [21]:
from pathlib import Path

import numpy as np
import tensorflow as tf
import yaml

from dvclive import Live
from dvclive.keras import DVCLiveCallback

params = Path("../params.yaml")
with open(params, "r") as file:
    params = yaml.safe_load(file)
DROPOUT = params["train"]["dropout"]
PATIENCE = params["train"]["patience"]
KERN_REG = params["train"]["kernel_regularizer"]
BATCH_NORMALIZATION = params["train"]["batch_normalization"]
BATCH_SIZE = params["train"]["batch_size"]
EPOCHS = params["train"]["epochs"]
LSTM_UNITS = params["train"]["lstm_units"]


def compile_and_fit(model, X_train, y_train, X_val, y_val, pitcher_name):
    optimizer = tf.keras.optimizers.Adam(
        learning_rate=1e-4,  # - learning_rate: controls how much to change the model in response to the estimated error each time the model weights are updated.
        clipnorm=1.0,  # - clipnorm: clips gradients by norm; helps prevent exploding gradients.
        weight_decay=1e-4,  # - weight_decay: adds a penalty to the loss function to prevent overfitting.
    )
    model.compile(
        optimizer=optimizer,
        loss="sparse_categorical_crossentropy",
        metrics=["sparse_categorical_accuracy"],
    )

    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor="val_sparse_categorical_accuracy",
            patience=PATIENCE,
            restore_best_weights=True,
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor="val_loss",
            factor=0.2,
            patience=PATIENCE,
            min_lr=1e-6,
        ),
        DVCLiveCallback(live=Live(f"dvclive/{pitcher_name}_logs")),
    ]

    history = model.fit(
        X_train,
        y_train,
        validation_data=(X_val, y_val),
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        callbacks=callbacks,
    )

    return history


def create_model(input_shape, num_classes):
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.InputLayer(shape=input_shape))
    model.add(
        tf.keras.layers.LSTM(
            LSTM_UNITS,
            return_sequences=True,
            dropout=DROPOUT,
            kernel_regularizer=tf.keras.regularizers.l2(KERN_REG),
        )
    )
    if BATCH_NORMALIZATION:
        model.add(tf.keras.layers.BatchNormalization())
    model.add(
        tf.keras.layers.LSTM(
            LSTM_UNITS,
            return_sequences=True,
            dropout=DROPOUT,
            kernel_regularizer=tf.keras.regularizers.l2(KERN_REG),
        )
    )
    if BATCH_NORMALIZATION:
        model.add(tf.keras.layers.BatchNormalization())
    model.add(
        tf.keras.layers.LSTM(
            LSTM_UNITS,
            dropout=DROPOUT,
            kernel_regularizer=tf.keras.regularizers.l2(KERN_REG),
        )
    )
    if BATCH_NORMALIZATION:
        model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.Dense(num_classes, activation="softmax"))
    return model


def calculate_class_weights(y):
    proportions = np.bincount(y) / len(y)
    for i, proportion in enumerate(proportions):
        if proportion == 0:
            proportions[i] = 1e-6
    inverseN = 1 / len(proportions)
    weights = [inverseN / proportion for proportion in proportions]
    return {i: w for i, w in enumerate(weights)}


In [22]:
df = pl.read_parquet(Path("../" + params["train"]["input_data_path"]))

In [23]:
logger = logging.getLogger("choo choo")
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
handler.setFormatter(
    logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
)
logger.addHandler(handler)

In [24]:
def create_sequences(X, y, time_steps):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X[i : (i + time_steps)]
        Xs.append(v)
        ys.append(y[i + time_steps])
    return np.array(Xs), np.array(ys)

In [25]:
def create_training_data(pitcher_df, features):
    # sort the data by game_date, game_pk, at_bat_number, pitch_number
    pitcher_df = pitcher_df.sort(
        [
            "game_date",
            "game_pk",
            "at_bat_number",
            "pitch_number",
        ],
        descending=False,
    )

    # select features and target variable for initial split of X and y
    X = pitcher_df.select(pl.col(features)).to_numpy()
    y = pitcher_df.select(pl.col("next_pitch")).to_numpy().ravel()

    # encode target variable
    label_encoder = LabelEncoder()
    y_encoded = label_encoder.fit_transform(y)
    y = y_encoded

    # declare time steps
    params_path = Path("../params.yaml")
    with open(params_path, "r") as file:
        params = yaml.safe_load(file)

    time_steps = params["train"]["time_steps"]

    X_seq, y_seq = create_sequences(X, y, time_steps)
    # convert to correct datatypes for model
    X_seq = X_seq.astype("float32")
    y_seq = y_seq.astype("int32")  # Ensure labels are integers

    # split into train, test, val
    train_split = params["train"]["train_split"]
    train_size = int(len(X_seq) * train_split)
    val_size = int(len(X_seq) * (1 - train_split) / 2)
    X_train, X_val, X_test = np.split(X_seq, [train_size, train_size + val_size])
    y_train, y_val, y_test = np.split(y_seq, [train_size, train_size + val_size])

    # scale features, no reason for minmax tbh, just read a stackoverflow that recommended it
    scaler = MinMaxScaler()
    n_samples, n_timesteps, n_features = X_train.shape
    X_train_reshaped = X_train.reshape(-1, n_features)
    X_train_scaled = scaler.fit_transform(X_train_reshaped)
    X_train_scaled = X_train_scaled.reshape(n_samples, n_timesteps, n_features)
    X_val_scaled = scaler.transform(X_val.reshape(-1, n_features)).reshape(X_val.shape)
    X_test_scaled = scaler.transform(X_test.reshape(-1, n_features)).reshape(
        X_test.shape
    )

    # logger.info("Unique labels in y_train:", list(np.unique(y_train)))
    # logger.info("Unique labels in y_val:", list(np.unique(y_val)))
    # logger.info("Unique labels in y_test:", list(np.unique(y_test)))

    return (
        X_train_scaled,
        y_train,
        X_val_scaled,
        y_val,
        X_test_scaled,
        y_test,
        label_encoder,
    )

In [26]:
def training_loop(df, params):
    pitcher_data = {}
    count = 0
    features = []
    features_path = Path("../" + params["train"]["features_path"])
    with open(features_path, "r") as f:
        for item in f.readlines():
            features.append(item.strip())
    start_time = time.time()
    for pitcher_df in df.group_by("pitcher"):
        pitcher_code = pitcher_df[0]
        pitcher_df = pitcher_df[1]

        pitcher_name = pitcher_df.select(pl.first("player_name")).item()
        num_pitches = len(pitcher_df)
        logger.info(
            f"Training model for pitcher: {pitcher_name} - {num_pitches} pitches"
        )
        X_train, y_train, X_val, y_val, X_test, y_test, label_encoder = (
            create_training_data(pitcher_df, features)
        )

        lstm_model = create_model(X_train.shape[1:], len(label_encoder.classes_))
        history = compile_and_fit(
            lstm_model,
            X_train,
            y_train,
            X_val,
            y_val,
            pitcher_name,
        )
        most_common_pitch_rate = pitcher_df.select(
            pl.col("pitch_type").value_counts().sort(descending=False).head(1)
        ).item()["count"] / len(pitcher_df)

        print(f"Most common pitch rate for {pitcher_name}: {most_common_pitch_rate}")
        test_loss, test_accuracy = lstm_model.evaluate(X_test, y_test)
        pitcher_data[pitcher_code] = {
            "model": lstm_model,
            "history": history,
            "test_loss": test_loss,
            "test_accuracy": test_accuracy,
            "total_pitches": len(y_test) + len(y_train) + len(y_val),
            "unique_classes": len(np.unique(y_train)),
            "player_name": pitcher_name,
            "X_test": X_test,
            "y_test": y_test,
            "label_encoder": label_encoder,
            "most_common_pitch_rate": most_common_pitch_rate,
            "performance_gain": (test_accuracy - most_common_pitch_rate) * 100,
        }
        logger.info(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}")
        logger.info(
            f" Accuracy Gained over guessing most common pitch for {pitcher_name}: {pitcher_data[pitcher_code]["performance_gain"]:.2f}%",
        )
        logger.info(
            f"Average Test Accuracy: {mean([pitcher_data[pitcher]['test_accuracy'] for pitcher in pitcher_data])*100:.2f}%"
        )

        logger.info(
            f"Average Performance Gained over guessing most common pitch: {mean([pitcher_data[pitcher]['performance_gain'] for pitcher in pitcher_data]):.2f}%"
        )

        count += 1
        logger.info(
            f'{count} of {len(df.select(pl.col("pitcher")).unique())}, {(count/len(df.select(pl.col("pitcher")).unique())) * 100:.2f}% done!'
        )
    end_time = time.time()
    logger.info(f"Training took {end_time - start_time} seconds")
    return pitcher_data

In [27]:
pitcher_data = training_loop(df, params)

2024-12-17 12:35:08,168 - choo choo - INFO - Training model for pitcher: Flexen, Chris - 2797 pitches
2024-12-17 12:35:08,168 - choo choo - INFO - Training model for pitcher: Flexen, Chris - 2797 pitches


Epoch 1/200
61/61 ━━━━━━━━━━━━━━━━━━━━ 2:08 2s/step - loss: 5.6107 - sparse_categorical_accuracy: 0.06 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 5.5781 - sparse_categorical_accuracy: 0.09 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 5.6052 - sparse_categorical_accuracy: 0.09 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 5.6181 - sparse_categorical_accuracy: 0.09 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 5.6189 - sparse_categorical_accuracy: 0.09 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 5.6164 - sparse_categorical_accuracy: 0.09 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 5.6127 - sparse_categorical_accuracy: 0.09 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 5.6077 - sparse_categorical_accuracy: 0.09 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 5.6028 - sparse_categorical_accuracy: 0.09 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 5.5974 - sparse_categorical_accuracy: 0.09 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 5.5927 - sparse_categorical_accuracy: 0.09 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 5.588

2024-12-17 12:36:23,167 - choo choo - INFO - Test Loss: 0.8006, Test Accuracy: 0.98
2024-12-17 12:36:23,167 - choo choo - INFO - Test Loss: 0.8006, Test Accuracy: 0.98
2024-12-17 12:36:23,170 - choo choo - INFO -  Accuracy Gained over guessing most common pitch for Flexen, Chris: 60.87%
2024-12-17 12:36:23,170 - choo choo - INFO -  Accuracy Gained over guessing most common pitch for Flexen, Chris: 60.87%
2024-12-17 12:36:23,171 - choo choo - INFO - Average Test Accuracy: 97.84%
2024-12-17 12:36:23,171 - choo choo - INFO - Average Test Accuracy: 97.84%
2024-12-17 12:36:23,173 - choo choo - INFO - Average Performance Gained over guessing most common pitch: 60.87%
2024-12-17 12:36:23,173 - choo choo - INFO - Average Performance Gained over guessing most common pitch: 60.87%
2024-12-17 12:36:23,182 - choo choo - INFO - 1 of 50, 2.00% done!
2024-12-17 12:36:23,182 - choo choo - INFO - 1 of 50, 2.00% done!
2024-12-17 12:36:23,204 - choo choo - INFO - Training model for pitcher: Blanco, Ronel

Epoch 1/200
63/63 ━━━━━━━━━━━━━━━━━━━━ 2:20 2s/step - loss: 5.7547 - sparse_categorical_accuracy: 0.12 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - loss: 5.6609 - sparse_categorical_accuracy: 0.15 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 5.6278 - sparse_categorical_accuracy: 0.15 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 5.6073 - sparse_categorical_accuracy: 0.15 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 5.6017 - sparse_categorical_accuracy: 0.15 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 5.5996 - sparse_categorical_accuracy: 0.14 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 5.5988 - sparse_categorical_accuracy: 0.14 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 5.5972 - sparse_categorical_accuracy: 0.14 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 5.5956 - sparse_categorical_accuracy: 0.13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 5.5931 - sparse_categorical_accuracy: 0.13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 5.5900 - sparse_categorical_accuracy: 0.13 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - loss: 5.586

2024-12-17 12:37:51,664 - choo choo - INFO - Test Loss: 0.5744, Test Accuracy: 0.99
2024-12-17 12:37:51,664 - choo choo - INFO - Test Loss: 0.5744, Test Accuracy: 0.99
2024-12-17 12:37:51,664 - choo choo - INFO -  Accuracy Gained over guessing most common pitch for Blanco, Ronel: 61.02%
2024-12-17 12:37:51,664 - choo choo - INFO -  Accuracy Gained over guessing most common pitch for Blanco, Ronel: 61.02%
2024-12-17 12:37:51,665 - choo choo - INFO - Average Test Accuracy: 98.45%
2024-12-17 12:37:51,665 - choo choo - INFO - Average Test Accuracy: 98.45%
2024-12-17 12:37:51,665 - choo choo - INFO - Average Performance Gained over guessing most common pitch: 60.95%
2024-12-17 12:37:51,665 - choo choo - INFO - Average Performance Gained over guessing most common pitch: 60.95%
2024-12-17 12:37:51,668 - choo choo - INFO - 2 of 50, 4.00% done!
2024-12-17 12:37:51,668 - choo choo - INFO - 2 of 50, 4.00% done!
2024-12-17 12:37:51,672 - choo choo - INFO - Training model for pitcher: Fedde, Erick 

...

2024-12-17 12:42:07,838 - choo choo - INFO - Test Loss: 0.7638, Test Accuracy: 0.94
2024-12-17 12:42:07,838 - choo choo - INFO - Test Loss: 0.7638, Test Accuracy: 0.94
2024-12-17 12:42:07,839 - choo choo - INFO -  Accuracy Gained over guessing most common pitch for Berríos, José: 75.46%
2024-12-17 12:42:07,839 - choo choo - INFO -  Accuracy Gained over guessing most common pitch for Berríos, José: 75.46%
2024-12-17 12:42:07,841 - choo choo - INFO - Average Test Accuracy: 76.33%
2024-12-17 12:42:07,841 - choo choo - INFO - Average Test Accuracy: 76.33%
2024-12-17 12:42:07,842 - choo choo - INFO - Average Performance Gained over guessing most common pitch: 48.25%
2024-12-17 12:42:07,842 - choo choo - INFO - Average Performance Gained over guessing most common pitch: 48.25%
2024-12-17 12:42:07,852 - choo choo - INFO - 7 of 50, 14.00% done!
2024-12-17 12:42:07,852 - choo choo - INFO - 7 of 50, 14.00% done!
2024-12-17 12:42:07,861 - choo choo - INFO - Training model for pitcher: Bibee, Tann

Epoch 1/200
70/70 ━━━━━━━━━━━━━━━━━━━━ 2:34 2s/step - loss: 5.4631 - sparse_categorical_accuracy: 0.21 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 5.6549 - sparse_categorical_accuracy: 0.16 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - loss: 5.6081 - sparse_categorical_accuracy: 0.16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 5.5798 - sparse_categorical_accuracy: 0.16 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 5.5646 - sparse_categorical_accuracy: 0.15 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 5.5612 - sparse_categorical_accuracy: 0.15 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 5.5547 - sparse_categorical_accuracy: 0.15 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 5.5491 - sparse_categorical_accuracy: 0.15 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 5.5434 - sparse_categorical_accuracy: 0.15 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 5.5359 - sparse_categorical_accuracy: 0.15 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - loss: 5.5282 - sparse_categorical_accuracy: 0.15 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step - loss: 5.519

2024-12-17 12:43:44,224 - choo choo - INFO - Test Loss: 1.1038, Test Accuracy: 0.87
2024-12-17 12:43:44,224 - choo choo - INFO - Test Loss: 1.1038, Test Accuracy: 0.87
2024-12-17 12:43:44,224 - choo choo - INFO -  Accuracy Gained over guessing most common pitch for Bibee, Tanner: 44.01%
2024-12-17 12:43:44,224 - choo choo - INFO -  Accuracy Gained over guessing most common pitch for Bibee, Tanner: 44.01%
2024-12-17 12:43:44,225 - choo choo - INFO - Average Test Accuracy: 77.65%
2024-12-17 12:43:44,225 - choo choo - INFO - Average Test Accuracy: 77.65%
2024-12-17 12:43:44,226 - choo choo - INFO - Average Performance Gained over guessing most common pitch: 47.72%
2024-12-17 12:43:44,226 - choo choo - INFO - Average Performance Gained over guessing most common pitch: 47.72%
2024-12-17 12:43:44,229 - choo choo - INFO - 8 of 50, 16.00% done!
2024-12-17 12:43:44,229 - choo choo - INFO - 8 of 50, 16.00% done!
2024-12-17 12:43:44,233 - choo choo - INFO - Training model for pitcher: Miller, Bry

Epoch 1/200


KeyboardInterrupt: 